# BaiscTick: Create Cluster: RDB
This notebook will create (start) an RDB cluster on a named database.

In [ ]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [ ]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env_kdb_1 import *

# Cluster names and database
from basictick_setup import *

SEC_THREADS='4'

# Cluster Settings

CODEBASE="basictick"
S3_CODE_PATH="code"

CAPACITY_CONFIG={ 'nodeCount': 1, 'nodeType': 'kx.s.xlarge'}
SAVEDOWN_CONFIG={ 'type': 'SDS01', 'size':100 }
DATABASE_CONFIG=[{ 'databaseName': DB_NAME }]
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }
INIT_SCRIPT=f'{CODEBASE}/rdbmkdb.q'

CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'dbname', 'value': DB_NAME}, 
    { 'key': 'codebase', 'value': CODEBASE}, 
#    { 'key': 'tphostfile', 'value': TP_HOSTFILE}, 
    {'key': 'tp', 'value': TP }, 
]

In [ ]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

## Check Database

In [ ]:
note_str = ""

try:
    c_set_list = client.list_kx_changesets(environmentId=ENV_ID, databaseName=DB_NAME)['kxChangesets']
except:
    note_str = "<<Could not get changesets>>"

print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"Changeset ({c['status']}): {c_set_id}: Created: {c['createdTimestamp']}")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

# Deploy code

In [ ]:
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
else:
    cp = f"""
zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
    
# execute the S3 copy
os.system(cp)

## Create Cluster

In [ ]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=RDB_CLUSTER_NAME,
    clusterType='RDB',
    releaseLabel = '1.0',
    databases=DATABASE_CONFIG,
    capacityConfiguration=CAPACITY_CONFIG,
    savedownStorageConfiguration=SAVEDOWN_CONFIG,
    clusterDescription="Created with create_RDB notebook",
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    commandLineArguments=CMD_ARGS,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [ ]:
resp

In [ ]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=RDB_CLUSTER_NAME)
print("** DONE **")

In [ ]:
cdf = get_clusters(client, environmentId=ENV_ID, clusterType="RDB")

display(cdf)

In [ ]:
print( f"Last Run: {datetime.datetime.now()}" )